In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [3]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\N1.xml,1093,1396,645,727
1,./images\N100.xml,134,301,312,350
2,./images\N101.xml,31,139,128,161
3,./images\N102.xml,164,316,216,243
4,./images\N103.xml,813,1067,665,724


In [4]:
import xml.etree.ElementTree as xet

In [5]:
filename = df['filepath'][0]
filename

'./images\\N1.xml'

In [6]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images',filename_image)
    return filepath_image

In [7]:
getFilename(filename)

'./images\\N1.jpeg'

In [8]:
image_path = list(df['filepath'].apply(getFilename))
image_path 

['./images\\N1.jpeg',
 './images\\N100.jpeg',
 './images\\N101.jpeg',
 './images\\N102.jpeg',
 './images\\N103.jpeg',
 './images\\N104.jpeg',
 './images\\N105.jpeg',
 './images\\N106.jpeg',
 './images\\N107.jpeg',
 './images\\N108.jpeg',
 './images\\N109.jpeg',
 './images\\N11.jpeg',
 './images\\N110.jpeg',
 './images\\N111.jpeg',
 './images\\N112.jpeg',
 './images\\N113.jpeg',
 './images\\N114.jpeg',
 './images\\N116.jpeg',
 './images\\N117.jpeg',
 './images\\N118.jpeg',
 './images\\N119.jpeg',
 './images\\N12.jpeg',
 './images\\N120.jpeg',
 './images\\N121.jpeg',
 './images\\N122.jpeg',
 './images\\N123.jpeg',
 './images\\N124.jpeg',
 './images\\N126.jpeg',
 './images\\N127.jpeg',
 './images\\N128.jpeg',
 './images\\N129.jpeg',
 './images\\N130.jpeg',
 './images\\N131.jpeg',
 './images\\N132.jpeg',
 './images\\N133.jpeg',
 './images\\N134.jpeg',
 './images\\N135.jpeg',
 './images\\N136.jpeg',
 './images\\N137.jpeg',
 './images\\N138.jpeg',
 './images\\N139.jpeg',
 './images\\N14.jpeg

#### verify image and output

In [9]:
file_path = image_path[0]
file_path 

'./images\\N1.jpeg'

In [10]:
img = cv2.imread(file_path)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
# 1093	1396	645	727
cv2.rectangle(img,(1093,645),(1396,727),(0,255,0),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Data Preprocessing

In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [13]:
labels = df.iloc[:,1:].values

In [14]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # normalization
    # normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # normalized output
    # -------------- append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [15]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [16]:
X.shape,y.shape

((225, 224, 224, 3), (225, 4))

In [17]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

### Deep Learning Model

In [18]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [19]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)


219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 359s 2us/step


In [20]:
# complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 73,663,490 (281.00 MB)

 Trainable params: 19,326,754 (73.73 MB)

 Non-trainable params: 54,336,736 (207.28 MB)

### model training

In [21]:
from tensorflow.keras.callbacks import TensorBoard

In [22]:
tfb = TensorBoard('object_detection')

In [23]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - loss: 0.1376 - val_loss: 0.1627
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 0.1510 - val_loss: 0.1475
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 0.1524 - val_loss: 0.1683
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - loss: 0.1624 - val_loss: 0.1505
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - loss: 0.1494 - val_loss: 0.1634
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - loss: 0.1520 - val_loss: 0.1473
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - loss: 0.1434 - val_loss: 0.1488
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - loss: 0.1377 - val_loss: 0.1595
Epoch 9/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - loss: 0.1461 - val_loss: 0.1471
Epoch 10/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - loss: 0.1375 - val_loss: 0.1423
Epoch 11/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - loss: 0.1438 - val_loss: 0.1458
Epoch 12/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - loss: 0.1

KeyboardInterrupt: 

In [23]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=200,
                    validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=101)

Epoch 102/200
18/18 [==============================] - 34s 2s/step - loss: 2.9122e-04 - val_loss: 0.0063
Epoch 103/200
18/18 [==============================] - 21s 1s/step - loss: 1.9947e-04 - val_loss: 0.0059
Epoch 104/200
18/18 [==============================] - 21s 1s/step - loss: 2.0441e-04 - val_loss: 0.0061
Epoch 105/200
18/18 [==============================] - 19s 1s/step - loss: 1.8242e-04 - val_loss: 0.0059
Epoch 106/200
18/18 [==============================] - 19s 1s/step - loss: 1.5497e-04 - val_loss: 0.0059
Epoch 107/200
18/18 [==============================] - 21s 1s/step - loss: 2.8745e-04 - val_loss: 0.0060
Epoch 108/200
18/18 [==============================] - 20s 1s/step - loss: 3.3103e-04 - val_loss: 0.0063
Epoch 109/200
18/18 [==============================] - 21s 1s/step - loss: 2.3359e-04 - val_loss: 0.0059
Epoch 110/200
18/18 [==============================] - 22s 1s/step - loss: 1.7057e-04 - val_loss: 0.0061
Epoch 111/200
18/18 [==============================] - 

18/18 [==============================] - 20s 1s/step - loss: 4.3210e-04 - val_loss: 0.0059
Epoch 178/200
18/18 [==============================] - 20s 1s/step - loss: 5.2341e-04 - val_loss: 0.0058
Epoch 179/200
18/18 [==============================] - 20s 1s/step - loss: 5.5556e-04 - val_loss: 0.0063
Epoch 180/200
18/18 [==============================] - 23s 1s/step - loss: 4.1865e-04 - val_loss: 0.0065
Epoch 181/200
18/18 [==============================] - 21s 1s/step - loss: 4.0438e-04 - val_loss: 0.0064
Epoch 182/200
18/18 [==============================] - 20s 1s/step - loss: 4.0499e-04 - val_loss: 0.0065
Epoch 183/200
18/18 [==============================] - 20s 1s/step - loss: 4.8212e-04 - val_loss: 0.0057
Epoch 184/200
18/18 [==============================] - 20s 1s/step - loss: 5.5655e-04 - val_loss: 0.0065
Epoch 185/200
18/18 [==============================] - 20s 1s/step - loss: 6.0796e-04 - val_loss: 0.0065
Epoch 186/200
18/18 [==============================] - 20s 1s/step - 

In [25]:
model.save('./models/object_detection.h5')